# Comparing typical antagonism curves from both models
The point is to show that the François 2013 model produce antagonism curves very far from the data. Use hand-picked, default parameters to illustrate how different the two models' antagonism curves are. 

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os, sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from utils.preprocess import write_conc_uM, michaelis_menten, inverse_michaelis_menten
from mcmc.utilities_tcr_car_antagonism import load_tcr_car_molec_numbers, prepare_car_antagonism_data
from models.tcr_car_akpr_model import (steady_akpr_i_receptor_types, steady_akpr_i_1ligand)
from models.tcr_car_francois2013 import (solution_francois2013_many_receptor_types, 
                            solution_francois2013_single_type, get_thresholds_francois2013)

In [ ]:
do_save_plots = False
do_save_outputs = False

mcmc_folder = os.path.join("..", "results", "mcmc")
res_folder = os.path.join("..", "results", "for_plots")
fig_folder = os.path.join("..", "figures", "model_comparison")

In [ ]:
def activation_function(x, h, pwr=1):
    return x**pwr / (x**pwr + h**pwr)

## Aesthetic parameters

In [ ]:
with open(os.path.join(res_folder, "perturbations_palette.json"), "r") as h:
    perturbations_palette = json.load(h)
sns.palplot(perturbations_palette.values())

In [ ]:
def change_log_ticks(axis, base=2, which="y"):
    if which == "y" or which == "both":
        ogyticks = axis.get_yticks()
        newyticks = list(np.unique([int(x) for x in ogyticks]))
        newyticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newyticks]
        axis.set_yticks(newyticks)
        axis.set_yticklabels(newyticklabels)
    if which == "x" or which == "both":
        ogxticks = axis.get_xticks()
        newxticks = list(np.unique([int(x) for x in ogxticks]))
        newxticklabels = [str(base)+'$^{'+str(x)+'}$' for x in newxticks]
        axis.set_xticks(newxticks)
        axis.set_xticklabels(newxticklabels)
    return axis

## Import external parameters: receptors and ligand numbers

In [ ]:
# Number of TCR and CAR per T cell, CD19 per tumor, pulse KD, peptide taus
molec_counts_fi = os.path.join("..", "data", 
    "surface_counts", "surface_molecule_summary_stats.h5")
mtc = "Geometric mean"
res = load_tcr_car_molec_numbers(molec_counts_fi, mtc, tcell_type="OT1_CAR", data_fold="../data/")
tcr_number, car_number, cd19_number = res[:3]
l_conc_mm_params, pep_tau_map = res[3:]

In [ ]:
def reverse_mm_fitted(x):
    return inverse_michaelis_menten(x, *l_conc_mm_params)

## Import and preprocess CAR-TCR antagonism data
Goal: have antagonism ratio at 1 nM and 1 uM TCR antigen density. 

In [ ]:
data_file_name = os.path.join("..", "data", "antagonism", "combined_OT1-CAR_antagonism.hdf")
data_f = pd.read_hdf(data_file_name)
chosen_fit_conc = "1uM"
data_prep = prepare_car_antagonism_data(data_f, l_conc_mm_params,
                pep_tau_map, cyto="IL-2", do_plot=False, dropn4=False,
                tcr_conc=["1uM", "1nM"], tcr_itams="10", car_itams="3", data_fold="data/")
df_ratios, df_ci_log2 = data_prep

# Extended François 2013 model with CARs and TCRs
The two types of receptors have different ligands, but they are coupled through the protein mediating the negative feedback. 

Here I am not going to reuse the extended model I had up to September 2022, because that one contained some ideas now in the new model, such as a negative feedback asymmetry between CAR and TCR, and sub-species $I^C$ and $I^T$. In fact, it's not really the François 2013 model anymore. Here, I really want to show that the original François 2013 model, minimally extended to CARs, does not work, and we need supplementary tweaks. The "best" extended François 2013 model with asymmetric $I$ is archived. For clarity, when cleaning up the final paper repository, remove the François 2013 model with asymmetric feedback. 

Also, no reference to my model in Gaud et al. 2022: this was complicated, with multiple normalized $C_n$s entering the output, to explain the 6Y-6F difference. Again, not what we want to focus on here. 


### Total output
Simply the sum of the TCR and CAR outputs, which could represent the total active ZAP-70 bound to either kind of receptor in the T cell:

$$ Z_{tot} = Z_{CAR} + Z_{TCR} $$

## Negative feedback asymmetry between CAR and TCR
I am not adding this in yet in the model, however, each receptor can be dephosphorylated by $I$ with a different rate $\gamma^i$. 



In [ ]:
# Define model rates
f2013_rates = [
    np.asarray([0.1, 0.001]),     # phi
    np.asarray([0.04, 0.0004]),    # b
    np.asarray([1.2e-6, 2e-7]),  # gamma
    np.asarray([1e-4, 1e-3])     # kappa
]
f2013_rsp = np.asarray([tcr_number, car_number])
f2013_iparams = [6e5, np.asarray([1000, car_number*100])]  # CAR very poor at activating I
f2013_nparams = [
    np.asarray([6, 3]), 
    np.asarray([3, 1])
]
f2013_tau_crits = [10.0, 700.0]
cd19_l_tau = [cd19_number, 500.0]

## Check model outputs of TCR and CAR separately

In [ ]:
tcr_rates = [a[0] for a in f2013_rates]
tcr_ip = [f2013_iparams[0], f2013_iparams[1][0]]
tcr_np = [a[0] for a in f2013_nparams]

car_rates = [a[1] for a in f2013_rates]
car_ip = [f2013_iparams[0], f2013_iparams[1][1]]
car_np = [a[1] for a in f2013_nparams]

l_range_tcr = np.logspace(0, 6, 101)
tau_range_tcr = np.asarray([1, 3, 5, 7, 9], dtype=np.float64)
l_range_car = np.logspace(1, 7, 101)
tau_range_car = tau_range_tcr * 100.0

# (ratep, taup, Lp, Rp, iparams, nparams, precision=1e-6
car_outputs = np.zeros([tau_range_car.size, l_range_car.size])
tcr_outputs = np.zeros([tau_range_tcr.size, l_range_tcr.size])
for i in range(len(tau_range_tcr)):
    for j in range(len(l_range_tcr)):
        complexes = solution_francois2013_single_type(tcr_rates, tau_range_tcr[i], l_range_tcr[j], 
                                                  f2013_rsp[0], tcr_ip, tcr_np)
        tcr_outputs[i, j] = complexes[-2]
        complexes = solution_francois2013_single_type(car_rates, tau_range_car[i], l_range_car[j], 
                                                  f2013_rsp[1], car_ip, car_np)
        car_outputs[i, j] = complexes[-2]

fig, axes = plt.subplots(1, 2)
axes = axes.flatten()
fig.set_size_inches(9, 3.5)
for i in range(len(tau_range_tcr)):
    axes[0].plot(l_range_tcr, tcr_outputs[i], label=tau_range_tcr[i])
    axes[1].plot(l_range_car, car_outputs[i], label=tau_range_car[i])
axes[0].set(xlabel=r"TCR antigen $L$", ylabel="Output", xscale="log", yscale="log")
axes[1].set(xlabel=r"CAR antigen $L$", ylabel="Output", xscale="log", yscale="log")
axes[0].legend(title=r"$\tau$")
axes[1].legend(title=r"$\tau$")
fig.tight_layout()
plt.show()
plt.close()

## Compute antagonism ratio

In [ ]:
# Compute antagonism ratio curves for a few different L_TCR, as a function of tau_TCR
def francois2013_antag_panel(ratesp, l_tau_car, Rsp, iparams, nparams, 
                mm_params, tau_cs, tcr_tau_max=10.0, tcr_conc_select=[1.0, 1e-3]):  # tcr_conc in uM
    # Prepare tcr l and tau (x axis and lines in plots)
    tcr_tau_range = np.linspace(0.001, tcr_tau_max, 101)
    tcr_l_select = np.asarray([michaelis_menten(a, *mm_params) for a in tcr_conc_select])
    #tcr_l_select = np.asarray([60000, 6000, 600], dtype=np.float64)
    
    # Compute thresholds
    tcr_car_threshs = get_thresholds_francois2013(ratesp, tau_cs, Rsp, iparams, nparams)
    # Compute CAR alone 
    car_rates = [a[1] for a in ratesp]
    car_ip = [iparams[0], iparams[1][1]]
    car_np = [a[1] for a in nparams]
    # (ratep, taup, Lp, Rp, iparams, nparams, precision=1e-6
    car_alone = solution_francois2013_single_type(car_rates, l_tau_car[1], l_tau_car[0], 
                                                  Rsp[1], car_ip, car_np)[car_np[0]]
    car_alone = activation_function(car_alone, tcr_car_threshs[1], pwr=1)
    # For each choice of condition, compute antagonism ratio
    ratios = np.zeros([tcr_l_select.size, tcr_tau_range.size])
    for i in range(tcr_l_select.size):
        lvec = np.asarray([tcr_l_select[i], l_tau_car[0]])
        for j in range(tcr_tau_range.size):
            tauvec = np.asarray([tcr_tau_range[j], l_tau_car[1]])
            #solution_francois2013_many_receptor_types(ratesp, tausp, Lsp, Rsp, iparams, nparams, precision=1e-6)
            complexes = solution_francois2013_many_receptor_types(ratesp, tauvec, lvec, Rsp, iparams, nparams)
            ztcr = activation_function(complexes[0][-1], tcr_car_threshs[0], pwr=1)
            zcar = activation_function(complexes[1][-1], tcr_car_threshs[1], pwr=1)
            ratios[i, j] = (ztcr + zcar) / car_alone
    return tcr_l_select, tcr_conc_select, tcr_tau_range, ratios

In [ ]:
res = francois2013_antag_panel(f2013_rates, cd19_l_tau, f2013_rsp, f2013_iparams, 
        f2013_nparams, l_conc_mm_params, f2013_tau_crits)
l_range, pulse_range, tau_range, model_ratios = res

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 2.5)
palet = {"1uM":(0., 0., 0., 1.), "1nM": perturbations_palette["AgDens"]}
#palet = sns.dark_palette(perturbations_palette["AgDens"], n_colors=len(pulse_range))[::-1]
for i in range(len(l_range)):
    conc_lbl = write_conc_uM(pulse_range[i])
    ax.plot(tau_range, model_ratios[i], label=conc_lbl, color=palet[conc_lbl])
ax.set(xlabel=r"TCR antigen model $\tau$ (s)", ylabel=r"FC$_{TCR \rightarrow CAR}$")
ax.set_yscale("log", base=2)
ax.axhline(1.0, ls="--", color="grey", lw=1.0)
ax.legend(title=r"TCR Ag Density", frameon=False, borderaxespad=0.1,)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(fig_folder, "francois2013_tcr_car_typical_antagonism.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
## Saving to disk for plotting elsewhere
# Compute more intermediate TCR Ag densities
illustrated_tcr_pulse_concs = np.logspace(-3, 0.0, 4)

res = francois2013_antag_panel(f2013_rates, cd19_l_tau, f2013_rsp, f2013_iparams, 
        f2013_nparams, l_conc_mm_params, f2013_tau_crits, tcr_conc_select=illustrated_tcr_pulse_concs)
l_range, pulse_range, tau_range, model_ratios = res

df_model = pd.DataFrame(model_ratios, 
             index=pd.MultiIndex.from_arrays([pulse_range, l_range], names=["pulse_concentration", "L"]), 
             columns=pd.Index(tau_range, name="tau"))

# Write these model curves to disk for plotting elsewhere
typical_curves_file = os.path.join(res_folder, "typical_tcr_car_model_curves.h5")
if do_save_outputs:
    df_model.to_hdf(typical_curves_file,  key="francois2013")

### Remark
The Francois 2013 model has some rightwards shift of the antagonism curve when the TCR antigen density is decreased, but the amplitude of antagonism depends on that parameter too strongly. 

In [ ]:
# Wrapper around solution_francois2013_receptor_types, to be minimized wrt tau_t
def log_total_output_francois2013(taut, lt, ratesp, cd19, rsp, iparams, nparams, tc_threshs):
    tauvec = np.asarray([taut, cd19[1]])
    lvec = np.asarray([lt, cd19[0]])
    complexes = solution_francois2013_many_receptor_types(ratesp, tauvec, lvec, 
                                        rsp, iparams, nparams)
    ztot = activation_function(complexes[0][-1], tc_threshs[0], pwr=1)
    ztot += activation_function(complexes[1][-1], tc_threshs[1], pwr=1)
    return np.log10(ztot)

def log_ilevel_francois2013(taut, lt, ratesp, cd19, rsp, iparams, nparams, tc_threshs):
    tauvec = np.asarray([taut, cd19[1]])
    lvec = np.asarray([lt, cd19[0]])
    complexes = solution_francois2013_many_receptor_types(ratesp, tauvec, lvec, 
                                        rsp, iparams, nparams)
    return np.log10(complexes[-1])

# Compute max antagonism amplitude and tau as a function of L^T, show it's nothing like data
def find_max_antagonism(lt, *output_args):
    # Find minimum tau
    res = sp.optimize.minimize_scalar(log_total_output_francois2013, 
                    bracket=[1e-6, 1.0, 20.0], bounds=[1e-6, 20.0],
                    args=(lt, *output_args))
    best_antag_tau = res.x
    
    # Compute antagonism amplitude at that tau
    output_at_max = 10.0**log_total_output_francois2013(best_antag_tau, lt, *output_args)
    
    # Also compute I at the max antagonism
    i_at_max = 10.0**log_ilevel_francois2013(best_antag_tau, lt, *output_args)
    
    # Take care of dividing by agonist alone outside of this function
    return best_antag_tau, output_at_max, i_at_max

In [ ]:
def francois2013_max_antagonism_curves(ratesp, l_tau_car, Rsp, iparams, nparams, mm_params, tau_cs):
    # Agonist alone and thresholds
    # Compute thresholds
    tcr_car_threshs = get_thresholds_francois2013(ratesp, tau_cs, Rsp, iparams, nparams)
    # Compute CAR alone 
    car_rates = [a[1] for a in ratesp]
    car_ip = [iparams[0], iparams[1][1]]
    car_np = [a[1] for a in nparams]
    # (ratep, taup, Lp, Rp, iparams, nparams, precision=1e-6
    car_alone = solution_francois2013_single_type(car_rates, l_tau_car[1], l_tau_car[0], 
                                                  Rsp[1], car_ip, car_np)[car_np[0]]
    car_alone = activation_function(car_alone, tcr_car_threshs[1], pwr=1)
    
    pulse_conc_range = np.logspace(0.0, -4.0, 101)
    ltrange = [michaelis_menten(l, *mm_params) for l in pulse_conc_range]
    min_taus, min_outputs, min_ilvls = [], [], []
   
    for l in ltrange:
        res = find_max_antagonism(l, ratesp, l_tau_car, Rsp, iparams, nparams, tcr_car_threshs)
        min_taus.append(res[0])
        min_outputs.append(res[1] / car_alone)
        min_ilvls.append(res[2])
    
    min_taus = np.asarray(min_taus)
    min_outputs = np.asarray(min_outputs)
    min_ilvls = np.asarray(min_ilvls)
    
    print("Finished computing the optimum curves")
    
    fig, axes = plt.subplots(3, sharex=True, sharey=False)
    x = pulse_conc_range
    axes[0].plot(x, min_taus)
    axes[1].plot(x, min_outputs)
    axes[2].plot(x, min_ilvls)
    #axes[0].plot(ltrange, min_taus)
    #axes[1].plot(ltrange, min_outputs)
    axes[0].set(ylabel=r"Best antagonist $\tau$", xscale="log")
    axes[1].set(ylabel="Maximal antagonism", xscale="log")
    axes[1].set_yscale("log", base=2)
    axes[2].set(ylabel="I at optimum", xscale="log", yscale="log", xlabel=r"TCR Antigen Density ($\mu$M)")
    fig.tight_layout()
    return pulse_conc_range, ltrange, min_taus, min_outputs, [fig, axes]

In [ ]:
res_ = francois2013_max_antagonism_curves(f2013_rates, cd19_l_tau, f2013_rsp, f2013_iparams, 
                                                    f2013_nparams, l_conc_mm_params, f2013_tau_crits)
pulse_conc_range, ltrange, min_taus, min_outputs, figaxes = res_
plt.show()
plt.close()

In [ ]:
# Compare scaling of max antagonism and peak position to data for 1 nM and 1 uM antagonists
most_antagonism_data = (df_ratios.groupby("TCR_Antigen_Density").min()
                        .rename(reverse_mm_fitted, axis=0, level="TCR_Antigen_Density"))
most_antagonism_peptides = (df_ratios.groupby("TCR_Antigen_Density").idxmin()
                        .rename(reverse_mm_fitted, axis=0, level="TCR_Antigen_Density")
                        .rename(write_conc_uM, axis=0, level="TCR_Antigen_Density"))
most_antagonism_ci = (df_ci_log2.loc[most_antagonism_peptides]
                        .rename(reverse_mm_fitted, axis=0, level="TCR_Antigen_Density")
                        .droplevel("TCR_Antigen"))

# Plotting in log scale, changing y ticks labels later
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 2.5)
ax.axhline(0.0, ls="--", color="grey", lw=1.0)
palet = {"1uM":(0., 0., 0., 1.), "1nM": perturbations_palette["AgDens"]}
markers = {"1uM": "o", "1nM": "s"}
# Francois 2013 model
ax.plot(pulse_conc_range, np.log2(min_outputs), label="Model", color="k")
# Data
for i, conc in enumerate(most_antagonism_data.index):
    conc_lbl = write_conc_uM(conc)
    clr = palet.get(conc_lbl)
    lbl = "Data " + conc_lbl
    ax.errorbar(conc, np.log2(most_antagonism_data.loc[conc]), yerr=most_antagonism_ci.loc[conc], 
               ls="none", marker=markers.get(conc_lbl), mfc=clr, mec=clr, ms=6, label=lbl, 
               color=clr)

conc_lbl = r"TCR Ag Density ($\mu$M)"
ax.set(xlabel=conc_lbl, ylabel=r"Peak antagonism FC$_{TCR \rightarrow CAR}$", xscale="log")
change_log_ticks(ax, base=2, which="y")
ax.set_ylim([ax.get_ylim()[0], 0.2])
ax.legend(frameon=False, borderaxespad=0.1, fontsize=9)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(fig_folder, "francois2013_tcr_car_peak_antagonism.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Concatenate some stuff for simplicity
# Get the tau of the most antagonizing peptide now that we have sliced with it
peak_peptide = most_antagonism_peptides.map(lambda x: x[1])
peak_peptide.name = "tau"
# Rename concentrations in data and ci now that we have sliced them
peak_amplitude = most_antagonism_data.rename(write_conc_uM, axis=0, level="TCR_Antigen_Density")
peak_amplitude_ci = most_antagonism_ci.rename(write_conc_uM, axis=0, level="TCR_Antigen_Density")

peak_info_data = pd.concat({
    "amplitude": peak_amplitude, 
    "amplitude_CI": peak_amplitude_ci, 
    "peptide": peak_peptide
    }, names=["measurement"], axis=1
)
# pulse_conc_range, ltrange, min_taus, min_outputs,
peak_info_model = pd.DataFrame(
    np.stack([min_outputs, min_taus], axis=1), 
    index=pd.MultiIndex.from_arrays([pulse_conc_range, ltrange], 
        names=["pulse_concentration", "L"]), 
    columns=pd.Index(["amplitude", "tau"], name="measurement")
)

# Save to disk
filename_most_antagonism = os.path.join(res_folder, "peak_antagonism_tcr_car.h5")
if do_save_outputs:
    peak_info_data.to_hdf(filename_most_antagonism, key="data")
    peak_info_model.to_hdf(filename_most_antagonism, key="model_francois2013")

# Improved AKPR model

Make the same kind of diagram for reasonable parameter values. Hopefully, it shows that the peak antagonism does shift with TCR antigen density, while the amplitude of the peak does not. 

In [ ]:
def split_tcr_car_params_revised(all_rates, ritot, nparams):
    """ 
    Args:
        all_rates: list of arrays of parameters, ordered
            phi_arr, kappa_arr, cmthresh, ithresh_arr,  k_arr, gamma_mat, psi_arr]
        ri_tots: [[R_TCR, R_CAR], I_tot]
        nparams: [Ns, ms, fs]
    """
    psi0_arr = all_rates[-1]
    tcr_rates = tuple(p[0] for p in all_rates[:5]) + (psi0_arr[0],)
    car_rates = tuple(p[1] for p in all_rates[:5]) + (psi0_arr[1],)

    # R, I params and n, m, f params
    tcr_ri = (ritot[0][0], ritot[1])
    car_ri = (ritot[0][1], ritot[1])
    tcr_nmf = tuple(p[0] for p in nparams)
    car_nmf = tuple(p[1] for p in nparams)
    return tcr_rates, car_rates, tcr_ri, car_ri, tcr_nmf, car_nmf

def get_thresholds_revised_akpr(all_rates, taucs, ritot, nparams):
    """ Compute the thresholds on TCR and CAR, given combined parameters """
    res = split_tcr_car_params_revised(all_rates, ritot, nparams)
    tcr_rates, car_rates, tcr_ri, car_ri, tcr_nmf, car_nmf = res
    tcr_thresh = steady_akpr_i_1ligand(tcr_rates, taucs[0], tcr_ri[0]*10.0, 
                                       tcr_ri, tcr_nmf, large_l=True)[tcr_nmf[0]]
    car_thresh = steady_akpr_i_1ligand(car_rates, taucs[1], car_ri[0]*10.0, 
                                       car_ri, car_nmf, large_l=True)[car_nmf[0]]
    return tcr_thresh, car_thresh

In [ ]:
# Compute antagonism ratio curves for a few different L_TCR, as a function of tau_TCR
def revised_akpr_antag_panel(ratesp, l_tau_car, ri_tots, nparams, mm_params, tau_cs, tcr_tau_max=10.0, 
                            tcr_conc_select=[1.0, 1e-3]):  # in uM
    # Prepare tcr l and tau (x axis and lines in plots)
    tcr_tau_range = np.linspace(0.001, tcr_tau_max, 101)
    tcr_l_select = np.asarray([michaelis_menten(a, *mm_params) for a in tcr_conc_select])
    #tcr_l_select = np.asarray([60000, 6000, 600], dtype=np.float64)
    
    # Compute thresholds
    tcr_car_threshs = get_thresholds_revised_akpr(ratesp, tau_cs, ri_tots, nparams)
    
    # Compute CAR alone 
    res = split_tcr_car_params_revised(ratesp, ri_tots, nparams)
    _, car_rates, _, car_ri, _, car_nmf = res
    car_alone = steady_akpr_i_1ligand(car_rates, l_tau_car[1], l_tau_car[0], 
                                                  car_ri, car_nmf)[car_nmf[0]]
    car_alone = activation_function(car_alone, tcr_car_threshs[1], pwr=2)
    # For each choice of condition, compute antagonism ratio
    ratios = np.zeros([tcr_l_select.size, tcr_tau_range.size])
    for i in range(tcr_l_select.size):
        lvec = np.asarray([tcr_l_select[i], l_tau_car[0]])
        for j in range(tcr_tau_range.size):
            tauvec = np.asarray([tcr_tau_range[j], l_tau_car[1]])
            #solution_francois2013_many_receptor_types(ratesp, tausp, Lsp, Rsp, iparams, nparams, precision=1e-6)
            complexes = steady_akpr_i_receptor_types(ratesp, tauvec, lvec, ri_tots, nparams)
            ztcr = activation_function(complexes[0][-1], tcr_car_threshs[0], pwr=2)
            zcar = activation_function(complexes[1][-1], tcr_car_threshs[1], pwr=2)
            ratios[i, j] = (ztcr + zcar) / car_alone
    return tcr_l_select, tcr_conc_select, tcr_tau_range, ratios

### Typical curves

In [ ]:
# Define model rates
akpr_rates = [ #phi_arr, kappa_arr, cmthresh, ithresh_arr,  k_arr, gamma_mat, psi_arr
    np.asarray([0.1, 0.001]),     # phis
    np.asarray([1e-4, 1e-3]),     # kappas
    np.asarray([1000.0, 7e5]),    # cmthreshs
    np.asarray([1e-5, 5e-4]),     # ithreshs
    np.asarray([1, 1]),           # k_Is
    np.asarray([[1.0, 0.1], [100.0, 1.0]]),  # gamma_mat
    np.asarray([1e-6, 5e-5])      # psi_0s
]
akpr_rip = [np.asarray([tcr_number, car_number]), 1.0]
akpr_nmfs = [
    np.asarray([6, 3]),  # N
    np.asarray([4, 2]),  # m
    np.asarray([1, 1])
]
akpr_tau_crits = [6.0, 400.0]

In [ ]:
l_range_akpr, pulse_range_akpr, tau_range_akpr, model_ratios_akpr = revised_akpr_antag_panel(
                                                    akpr_rates, cd19_l_tau, akpr_rip, akpr_nmfs, 
                                                    l_conc_mm_params, akpr_tau_crits)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 2.5)
palet = {"1uM":(0., 0., 0., 1.), "1nM": perturbations_palette["AgDens"]}
for i in range(len(l_range_akpr)):
    conc_lbl = write_conc_uM(pulse_range_akpr[i])
    ax.plot(tau_range_akpr, model_ratios_akpr[i], label=conc_lbl, color=palet[conc_lbl])
ax.set(xlabel=r"TCR antigen model $\tau$ (s)", ylabel=r"FC$_{TCR \rightarrow CAR}$")
ax.set_yscale("log", base=2)
ax.axhline(1.0, ls="--", color="grey", lw=1.0)
ax.legend(title=r"TCR Ag Density", frameon=False, borderaxespad=0.1, handlelength=1.5)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(fig_folder, "revised_akpr_tcr_car_typical_antagonism.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
## Saving to disk for plotting elsewhere
# Compute more intermediate TCR Ag densities
#illustrated_tcr_pulse_concs = np.logspace(-3, 0.0, 4)  # Same as other model

res = revised_akpr_antag_panel(akpr_rates, cd19_l_tau, akpr_rip, akpr_nmfs, 
            l_conc_mm_params, akpr_tau_crits, tcr_conc_select=illustrated_tcr_pulse_concs)
l_range, pulse_range, tau_range, model_ratios = res

df_model = pd.DataFrame(model_ratios, 
             index=pd.MultiIndex.from_arrays([pulse_range, l_range], names=["pulse_concentration", "L"]), 
             columns=pd.Index(tau_range, name="tau"))

# Write these model curves to disk for plotting elsewhere
# typical_curves_file = "results/for_plots/typical_tcr_car_model_curves.h5"  # Same as other model
if do_save_outputs:
    df_model.to_hdf(typical_curves_file,  key="revised_akpr")

### Antagonism peak amplitude and position as a function of TCR Ag density

In [ ]:
# Wrapper around steady_akpr_i_receptor_types, to be minimized wrt tau_t
def log_total_output_revised_akpr(taut, lt, ratesp, cd19, ri_tots, nparams, tc_threshs):
    tauvec = np.asarray([taut, cd19[1]])
    lvec = np.asarray([lt, cd19[0]])
    complexes = steady_akpr_i_receptor_types(ratesp, tauvec, lvec, ri_tots, nparams)
    ztot = activation_function(complexes[0][-1], tc_threshs[0], pwr=2)
    ztot += activation_function(complexes[1][-1], tc_threshs[1], pwr=2)
    return np.log10(ztot)

# Compute max antagonism amplitude and tau as a function of L^T, show it's nothing like data
def find_max_antagonism_akpr(lt, *output_args):
    # Find minimum tau
    res = sp.optimize.minimize_scalar(log_total_output_revised_akpr, 
                    bracket=[1e-6, 1.0, 20.0], bounds=[1e-6, 20.0],
                    args=(lt, *output_args))
    best_antag_tau = res.x
    
    # Compute antagonism amplitude at that tau
    output_at_max = 10.0**log_total_output_revised_akpr(best_antag_tau, lt, *output_args)
    
    # Take care of dividing by agonist alone outside of this function
    return best_antag_tau, output_at_max

In [ ]:
def revised_akpr_max_antagonism_curves(ratesp, l_tau_car, ri_tots, nparams, mm_params, tau_cs):
    # Agonist alone and thresholds
    # Compute thresholds
    tcr_car_threshs = get_thresholds_revised_akpr(ratesp, tau_cs, ri_tots, nparams)
    
    # Compute CAR alone 
    res = split_tcr_car_params_revised(ratesp, ri_tots, nparams)
    _, car_rates, _, car_ri, _, car_nmf = res
    car_alone = steady_akpr_i_1ligand(car_rates, l_tau_car[1], l_tau_car[0], 
                                                  car_ri, car_nmf)[car_nmf[0]]
    car_alone = activation_function(car_alone, tcr_car_threshs[1], pwr=2)
    
    pulse_conc_range = np.logspace(0.0, -4.0, 101)
    ltrange = [michaelis_menten(l, *mm_params) for l in pulse_conc_range]
    min_taus, min_outputs, min_ilvls = [], [], []
   
    for l in ltrange:
        res = find_max_antagonism_akpr(l, ratesp, l_tau_car, ri_tots, nparams, tcr_car_threshs)
        min_taus.append(res[0])
        min_outputs.append(res[1] / car_alone)
    
    min_taus = np.asarray(min_taus)
    min_outputs = np.asarray(min_outputs)
    
    print("Finished computing the optimum curves")
    
    fig, axes = plt.subplots(2, sharex=True, sharey=False)
    x = pulse_conc_range
    axes[0].plot(x, min_taus)
    axes[1].plot(x, min_outputs)
    #axes[0].plot(ltrange, min_taus)
    #axes[1].plot(ltrange, min_outputs)
    axes[0].set(ylabel=r"Best antagonist $\tau$", xscale="log")
    axes[1].set(ylabel="Maximal antagonism", xscale="log", xlabel=r"TCR Antigen Density ($\mu$M)")
    axes[1].set_yscale("log", base=2)
    fig.tight_layout()
    return pulse_conc_range, ltrange, min_taus, min_outputs, [fig, axes]

In [ ]:
res_ = revised_akpr_max_antagonism_curves(akpr_rates, cd19_l_tau, akpr_rip, akpr_nmfs,
                                                    l_conc_mm_params, akpr_tau_crits)
pulse_conc_range_akpr, ltrange_akpr, min_taus_akpr, min_outputs_akpr, figaxes = res_
plt.show()
plt.close()

In [ ]:
# Compare scaling of max antagonism and peak position to data for 1 nM and 1 uM antagonists
# Already done: most_antagonism_data, most_antagonism_peptides, most_antagonism_ci

# Plotting in log scale, changing y ticks labels later
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 2.5)
ax.axhline(0.0, ls="--", color="grey", lw=1.0)
palet = {"1uM":(0., 0., 0., 1.), "1nM": perturbations_palette["AgDens"]}
markers = {"1uM": "o", "1nM": "s"}
# Revised AKPR
ax.plot(pulse_conc_range_akpr, np.log2(min_outputs_akpr), label="Model", color="k")
# Data
for i, conc in enumerate(most_antagonism_data.index):
    conc_lbl = write_conc_uM(conc)
    clr = palet.get(conc_lbl)
    lbl = "Data " + conc_lbl
    ax.errorbar(conc, np.log2(most_antagonism_data.loc[conc]), yerr=most_antagonism_ci.loc[conc], 
               ls="none", marker=markers.get(conc_lbl), mfc=clr, mec=clr, ms=6, label=lbl, 
               color=clr)

conc_lbl = r"TCR Ag Density ($\mu$M)"
ax.set(xlabel=conc_lbl, ylabel=r"Peak antagonism FC$_{TCR \rightarrow CAR}$", xscale="log")
change_log_ticks(ax, base=2, which="y")
ax.set_ylim([ax.get_ylim()[0], 0.2])
ax.legend(frameon=False, borderaxespad=0.1, fontsize=9)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
fig.tight_layout()
if do_save_plots:
    fig.savefig(os.path.join(fig_folder, "revised_akpr_tcr_car_peak_antagonism.pdf"), 
               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Saving to disk
# No need to save the data again, it's the same
# Concatenate some stuff for simplicity

# pulse_conc_range, ltrange, min_taus, min_outputs,
peak_info_model = pd.DataFrame(
    np.stack([min_outputs_akpr, min_taus_akpr], axis=1), 
    index=pd.MultiIndex.from_arrays([pulse_conc_range_akpr, ltrange_akpr], 
        names=["pulse_concentration", "L"]), 
    columns=pd.Index(["amplitude", "tau"], name="measurement")
)

# Save to disk
#filename_most_antagonism = "results/for_plots/peak_antagonism_tcr_car.h5"  # Same as before
if do_save_outputs:
    peak_info_model.to_hdf(filename_most_antagonism, key="model_revised_akpr")